# Focusing on Fine-Tuning Mistral 7B

In [3]:
from dotenv import load_dotenv
from snowflake.snowpark.session import Session
import os
import json

load_dotenv()

connection_params = {
  "account": os.getenv("SNOWFLAKE_ACCOUNT"),
  "user": os.getenv("SNOWFLAKE_USER"),
  "password": os.getenv("SNOWFLAKE_USER_PASSWORD"),
  "role": os.getenv("SNOWFLAKE_ROLE"),
  "database": os.getenv("SNOWFLAKE_DATABASE"),
  "schema": os.getenv("SNOWFLAKE_SCHEMA"),
  "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE")
}

# Create the Snowpark session
snowpark_session = Session.builder.configs(connection_params).create()

In [3]:
# JSON file paths
json_files = {
    "constitution_qa": "dataset/constitution_qa.json",
    "crpc_qa": "dataset/crpc_qa.json",
    "ipc_qa": "dataset/ipc_qa.json"
}

In [7]:
import snowflake
def load_json_to_snowflake(table_name, file_path):
    # Connect to Snowflake
    conn = snowflake.connector.connect(**connection_params)
    cursor = conn.cursor()

    # Open the JSON file and load data
    with open(file_path, "r") as file:
        data = json.load(file)

    # Prepare and execute insert queries
    insert_query = f"INSERT INTO {table_name} (question, answer) VALUES (%s, %s)"
    for entry in data:
        cursor.execute(insert_query, (entry["question"], entry["answer"]))

    # Close the connection
    cursor.close()
    conn.close()
    print(f"Data loaded successfully into {table_name}.")


In [18]:
import pandas as pd

In [19]:
df = pd.read_json('dataset/constitution_qa.json')
df.to_csv('csv/constitution_qa.csv', encoding='utf-8', index=False)

In [20]:
df = pd.read_json('dataset/crpc_qa.json')
df.to_csv('csv/crpc_qa.csv', encoding='utf-8', index=False)

In [21]:
df = pd.read_json('dataset/ipc_qa.json')
df.to_csv('csv/ipc_qa.csv', encoding='utf-8', index=False)

## Testing with a prompt (Not yet fine-tuned)

In [2]:
from snowflake.cortex import Complete
print(Complete("mistral-7b", "What does Act 108A in Indian law discuss? Answer in one line"))

 Act 108A in Indian law is about the Prevention of Atrocities (Amendment) Act, 2015, which extends the provisions of the Scheduled Castes and Scheduled Tribes (Prevention of Atrocities) Act, 1989, to cover not just the scheduled castes and scheduled tribes but also the other backward classes.


In [4]:
df_merged_qa = snowpark_session .table('merged_qa')

In [6]:
df_merged_qa

In [8]:
import pandas as pd
import snowflake.connector
from llama_index.core import Document

In [12]:
conn = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_USER_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA")
)

In [13]:
query = "SELECT question, answer, qa FROM merged_qa"
df_merged_qa = pd.read_sql(query, conn)

conn.close()

/tmp/ipykernel_218841/3734711213.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_merged_qa = pd.read_sql(query, conn)


In [14]:
df_merged_qa

,QUESTION,ANSWER,QA
0,What is India according to the Union and its T...,"India, that is Bharat, shall be a Union of Sta...",QUESTION: What is India according to the Union...
1,"How is India, that is Bharat, defined in terms...","India, that is Bharat, is defined as a Union o...","QUESTION: How is India, that is Bharat, define..."
2,What does the territory of India comprise of?,The territory of India shall comprise the terr...,QUESTION: What does the territory of India com...
3,"What does the territory of a country, such as ...",The territory of a country like India comprise...,QUESTION: What does the territory of a country...
4,Who has the authority to admit or establish ne...,"Parliament may by law admit into the Union, or...",QUESTION: Who has the authority to admit or es...
...,...,...,...
14538,What came into effect on 1-1-1956?,The substitution for 'transportation',QUESTION: What came into effect on 1-1-1956? A...
14539,"What was substituted by s. 117 and the Sch., i...",Certain words,QUESTION: What was substituted by s. 117 and t...
14540,"What did s. 117 and the Sch., ibid. substitute?",Certain words,"QUESTION: What did s. 117 and the Sch., ibid. ..."
14541,When did the substitution for certain words co...,1-1-1956,QUESTION: When did the substitution for certai...


In [21]:

from llama_index.core import Document

# Assuming df_merged_qa has the columns 'question', 'answer', and 'qa' (combined)
documents = []

# Iterate over the DataFrame rows to create Documents
for _, row in df_merged_qa.iterrows():
     
    # Create a new Document with cleaned text
    doc = Document(
        text=row["QA"],   
        metadata=row.to_dict()  
    )
    
    # Append the cleaned document to the list
    documents.append(doc)


In [22]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SemanticSplitterNodeParser

embed_model = HuggingFaceEmbedding("Snowflake/snowflake-arctic-embed-m")

splitter = SemanticSplitterNodeParser(
  buffer_size=1, breakpoint_percentile_threshold=85, embed_model=embed_model
) 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/85.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [23]:
from llama_index.core.ingestion import IngestionPipeline

cortex_search_pipeline = IngestionPipeline(
  transformations=[
    splitter,
  ],
)

results = cortex_search_pipeline.run(show_progress=True, documents=documents)

Parsing nodes:   0%|          | 0/14543 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 